### Data Ingestion: Workflow

We will be following Workflow defined in the README File to complete Data Ingestion.

In [21]:
import os
import sys

In [22]:
sys.path.append(os.path.abspath("src"))

In [23]:
os.chdir('M:\\6. Personal Projects\\4. End to End NLP Text Summarization Using HuggingFace\\nlp_text_summarization_huggingface')

In [24]:
print("CWD:", os.getcwd())
print("Python Path:", sys.path)

CWD: M:\6. Personal Projects\4. End to End NLP Text Summarization Using HuggingFace\nlp_text_summarization_huggingface
Python Path: ['c:\\Users\\Maisam\\anaconda3\\envs\\textS\\python38.zip', 'c:\\Users\\Maisam\\anaconda3\\envs\\textS\\DLLs', 'c:\\Users\\Maisam\\anaconda3\\envs\\textS\\lib', 'c:\\Users\\Maisam\\anaconda3\\envs\\textS', '', 'c:\\Users\\Maisam\\anaconda3\\envs\\textS\\lib\\site-packages', 'c:\\Users\\Maisam\\anaconda3\\envs\\textS\\lib\\site-packages\\win32', 'c:\\Users\\Maisam\\anaconda3\\envs\\textS\\lib\\site-packages\\win32\\lib', 'c:\\Users\\Maisam\\anaconda3\\envs\\textS\\lib\\site-packages\\Pythonwin', 'c:\\Users\\Maisam\\anaconda3\\envs\\textS\\lib\\site-packages\\setuptools\\_vendor', 'M:\\6. Personal Projects\\4. End to End NLP Text Summarization Using HuggingFace\\nlp_text_summarization_huggingface\\src']


##### Creating Entity

In [43]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

#### Creating Configuration Manager

In [44]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [46]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
            ):

            self.config =   read_yaml(config_filepath)
            self.params =   read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

In [47]:
import urllib.request as request
import zipfile
from src.textSummarizer.logging import logger
from src.textSummarizer.utils.common import get_size

In [48]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url, 
                filename = self.config.local_data_file
                )
        
            logger.info(f"{filename} Downloaded! With Following Info: \n{headers}")
        else:
            logger.info(f"File Already Exists! Size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        extracts zip file in given directory
        funtion returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs (unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [49]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-13 18:11:39,226: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-13 18:11:39,228: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-13 18:11:39,229: INFO: common: created directory at: artifacts]
[2025-01-13 18:11:39,231: INFO: common: created directory at: artifacts/data_ingestion]
[2025-01-13 18:11:40,557: INFO: 3795769655: artifacts/data_ingestion/data.zip Downloaded! With Following Info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 45F8:60888:5E7D9B:7E1A77:6785575A
Accept-Ranges: bytes
Date: Mon, 13 Jan 2025 18:11:40 GMT
Via: 1.1 varnish
X-Served-By: cache-lhr-egll1980082-LHR
X-